# Packages

In [48]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MultiLabelBinarizer

# # Data Profiling

In [49]:
df = pd.read_excel(
    "/Users/yola.kamalita/Documents/Project/market-basket-analysis/dataset/online_retail_II.xlsx",
    engine="openpyxl"
)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [51]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [52]:
df['Country'].value_counts()

Country
United Kingdom          485852
EIRE                      9670
Germany                   8129
France                    5772
Netherlands               2769
Spain                     1278
Switzerland               1187
Portugal                  1101
Belgium                   1054
Channel Islands            906
Sweden                     902
Italy                      731
Australia                  654
Cyprus                     554
Austria                    537
Greece                     517
United Arab Emirates       432
Denmark                    428
Norway                     369
Finland                    354
Unspecified                310
USA                        244
Japan                      224
Poland                     194
Malta                      172
Lithuania                  154
Singapore                  117
RSA                        111
Bahrain                    107
Canada                      77
Hong Kong                   76
Thailand                    76


In [53]:
df["StockCode"].nunique()

4632

In [54]:
df["Description"].nunique()

4681

# Data Preprocessing

In [55]:
# Remove NAs

df_prep = df.dropna()

In [56]:
# Remove Quantity <= 0

df_prep = df_prep[df_prep['Quantity'] > 0]

In [57]:
# Filter out non-UK transactions

df_prep = df_prep[df_prep['Country'] == 'United Kingdom']

In [58]:
# Remove Trailing Spaces

df_prep['Description'] = df_prep['Description'].apply(lambda x: x.rstrip())

In [60]:
# Check rows with remaining trailing spaces

df_prep[df_prep['Description'].str.endswith(" ")]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


## Create new product groups

Hypothesis: The current product description is too detailed. Make it more general might be more useful to find the list of products to be cross-selling.

In [61]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yola.kamalita/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yola.kamalita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yola.kamalita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Text Embeddings

# Load BERT-based model
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('all-mpnet-base-v2')

# Product Description
products = df_prep['Description'].unique().tolist()

# Convert Product Description into embeddings
embeddings = model.encode(products)


In [149]:
# Clustering Product Description into Product Groups

# Compute cosine similarity
cosine_sim = cosine_similarity(embeddings)

# Perform clustering
clustering = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='complete', distance_threshold=0.5)
labels = clustering.fit_predict(1 - cosine_sim)  # Convert similarity to distance

# Print grouped products
product_groups = {}
for i, label in enumerate(labels):
    product_groups.setdefault(label, []).append(products[i])

/Users/yola.kamalita/anaconda3/envs/self_learning_projects/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [151]:
product_groups[0]

['S/15 SILVER GLASS BAUBLES IN BAG',
 'CLEAR COLOUR GLASS GEMS IN BAG',
 'ASSORTED COLOUR SILK GLASSES CASE',
 'DEEP BLUE GLASS GEMS IN BAG',
 'SKY BLUE COLOUR GLASS GEMS IN BAG',
 'BLUE GLASS GEMS IN BAG']

In [152]:
product_groups[100]

['FROG CANDLE',
 'JASMINE VOTIVE CANDLE',
 'LILAC VOTIVE CANDLE',
 'COFFEE SCENT PILLAR CANDLE',
 'FENG SHUI PILLAR CANDLE',
 'BLUE POT PLANT CANDLE',
 'CINNAMON SCENTED VOTIVE CANDLE',
 'STRAWBRY SCENTED VOTIVE CANDLE',
 'EAU DE NIL LOVE BIRD CANDLE',
 'IVORY LOVE BIRD CANDLE',
 'VANILLA SCENTED VOTIVE CANDLE',
 'YELLOW POT PLANT CANDLE',
 'OPIUM SCENTED VOTIVE CANDLE',
 'PINK POT PLANT CANDLE',
 'BLACK LOVE BIRD CANDLE',
 'PINK LOVE BIRD CANDLE']

In [153]:

# Naming the Product Group

# Stopwords to ignore
stop_words = set(stopwords.words("english")).union({"by", "for", "with", "the", "and", "of", "a", "to", "on", "in"})

# Initialize NLTK's WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess text (tokenization, stopword removal, lemmatization)
def preprocess_text(text):
    words = word_tokenize(text.lower())  # Convert to lowercase & tokenize
    words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]  # Lemmatize & remove stopwords
    return " ".join(words)

# Function to get representative name using TF-IDF
def get_representative_name(product_list):
    processed_products = [preprocess_text(p) for p in product_list]  # Preprocess product names
    
    # Compute TF-IDF scores
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(processed_products)
    
    # Find the highest scoring words
    feature_names = vectorizer.get_feature_names_out()
    scores = np.array(tfidf_matrix.sum(axis=0)).flatten()  # Sum TF-IDF scores per word
    top_words = [feature_names[i] for i in scores.argsort()[-3:]]  # Select top 3 words
    
    return " ".join(top_words).title()  # Format as title case

# Get representative names
representative_names = {group: get_representative_name(items) for group, items in product_groups.items()}

In [ ]:
representative_names[0]

'Gem Glass Bag'

In [155]:
representative_names[100]

'Scented Votive Candle'

In [156]:
# Convert Dict to DataFrame
product_groups_df = pd.DataFrame(list(product_groups.items()), columns=['Description Group', 'Description'])
representative_names_df = pd.DataFrame(list(representative_names.items()), columns=['Description Group', 'Description New'])

# Explode list into separate rows
product_groups_df = product_groups_df.explode('Description').reset_index(drop=True)
product_groups_df = pd.merge(product_groups_df, representative_names_df, on="Description Group", how="inner") 

In [157]:
product_groups_df

,Description Group,Description,Description New
0,104,15CM CHRISTMAS GLASS BALL 20 LIGHTS,Flock Ball Christmas
1,104,4 PINK FLOCK CHRISTMAS BALLS,Flock Ball Christmas
2,104,4 GOLD FLOCK CHRISTMAS BALLS,Flock Ball Christmas
3,177,PINK CHERRY LIGHTS,Flamingo Cherry Light
4,177,WHITE CHERRY LIGHTS,Flamingo Cherry Light
...,...,...,...
4396,679,BAKING MOULD CHOCOLATE CUPCAKES,Baking Mould Chocolate
4397,679,BAKING MOULD ROSE WHITE CHOCOLATE,Baking Mould Chocolate
4398,679,BAKING MOULD ROSE MILK CHOCOLATE,Baking Mould Chocolate
4399,679,BAKING MOULD CHOCOLATE CUP CAKES,Baking Mould Chocolate


## List of Product Group (New)

In [158]:
# Create List of Products for each Invoice

df_prep_merged = pd.merge(df_prep, product_groups_df, on="Description", how="inner") 
df_trx = df_prep_merged.groupby("Invoice")["Description New"].apply(list).reset_index()

In [159]:
df_trx.head(10)

,Invoice,Description New
0,489434,"[Flock Ball Christmas, Flamingo Cherry Light, ..."
1,489435,"[White Cat Bowl, Ball Design Dog, Heart Measur..."
2,489436,"[Mat Black Flock, Block Building Word, Block B..."
3,489437,"[Christmas Heart Decoration, Christmas Heart D..."
4,489438,"[Dinosaur Set Writing, Flower Animal Sticker, ..."
5,489440,"[White Cat Bowl, Ball Design Dog]"
6,489441,"[Spot Bird Decoration, Baking Retrospot Set, L..."
7,489442,"[Jack Rose Union, Water Hot Bottle, Heart Ivor..."
8,489443,"[Blue Washing Glove, Red Towel Tea, Record Sin..."
9,489445,"[Flower Hanging Heart, Flower Hanging Heart, S..."


In [160]:
df_trx = df_trx.set_index('Invoice')

df_trx.head()

,Description New
Invoice,
489434,"[Flock Ball Christmas, Flamingo Cherry Light, ..."
489435,"[White Cat Bowl, Ball Design Dog, Heart Measur..."
489436,"[Mat Black Flock, Block Building Word, Block B..."
489437,"[Christmas Heart Decoration, Christmas Heart D..."
489438,"[Dinosaur Set Writing, Flower Animal Sticker, ..."


In [161]:
# Encode List of Products for each Invoice

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Transform the column
df_trx_encode = mlb.fit_transform(df_trx["Description New"])

# Convert to DataFrame with proper column names
df_trx_encode = pd.DataFrame(df_trx_encode, columns=mlb.classes_, index=df_trx.index)

In [162]:
df_trx_encode.head()

,Acrylic Bangle Faceted,Acrylic Geometric Lamp,Aid First Tin,Airline Vintage Bag,Alarm Bakelike Clock,Alphabet Iron Patch,Animal Crocheted Japanese,Animal Farm Felt,Ant Bracelet Boudicca,Antique Edwardian Dresser,...,Woven Cover Cushion,Wrap Apple Red,Wrap Dolly Girl,Wrap London Love,Wrap Red Retrospot,Wreath Gingham Heart,Writing Balloon Set,Yellow Birdfeeder Chalet,Zinc Metal Heart,Zinc Stick Candle
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [163]:
# Number of transactions per product
df_trx_encode.sum().head(30).sort_values(ascending=False)

Antique Wood White            2037
Aid First Tin                  528
Assorted Frutti Tutti          456
Airline Vintage Bag            450
Assorted Cone Party            434
Assorted Magnet Fridge         393
Babushka Doorstop Gingham      352
Assorted Colour Flower         206
Alarm Bakelike Clock           200
Assorted Colour Teaspoon       176
Animal Crocheted Japanese      157
As Col Sand                    155
Assorted Crawlies Creepy       143
Assorted Circular Mobile       130
Asstd Design Pen               122
Assorted Floral Secateurs      110
Alphabet Iron Patch             80
Ant Bracelet Boudicca           57
Animal Farm Felt                53
Acrylic Geometric Lamp          41
Assorted Bucket Farmyard        28
Baby Bib Carousel               25
Antique Edwardian Dresser       20
Army Camo Tape                  16
Acrylic Bangle Faceted          15
Art Canvas Picture              13
Artiifcial Flower Foxglove      12
Asstd Col                        7
Arboretum English La

# Modelling

In [164]:
from mlxtend.frequent_patterns import apriori

In [167]:
frequent_itemsets = apriori(df_trx_encode, min_support=0.05, use_colnames=True) 

/Users/yola.kamalita/anaconda3/envs/self_learning_projects/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [168]:
frequent_itemsets.sort_values(by='support', ascending=False)

,support,itemsets
21,0.305041,(Glass Hanging Holder)
58,0.169240,(Water Hot Bottle)
30,0.159703,(Pack Case Cake)
29,0.149427,(Oval Trinket Box)
59,0.134211,(Way Sign Metal)
...,...,...
8,0.050812,(Card Heart Holder)
64,0.050812,"(Bag Red Retrospot, Spotty Red Bag)"
70,0.050812,"(Glass Hanging Holder, Star Heart Wicker)"
54,0.050358,(Storage Bag Retrospot)


In [169]:
frequent_itemsets[frequent_itemsets["itemsets"].apply(len) >= 2]

,support,itemsets
63,0.063018,"(Glass Hanging Holder, Antique Wood White)"
64,0.050812,"(Bag Red Retrospot, Spotty Red Bag)"
65,0.051209,"(Glass Hanging Holder, Block Building Word)"
66,0.050982,"(Spotty Red Bag, Design Bag Suki)"
67,0.059157,"(Glass Hanging Holder, Laundry Metal Sign)"
68,0.059271,"(Oval Trinket Box, Glass Hanging Holder)"
69,0.059896,"(Pack Case Cake, Glass Hanging Holder)"
70,0.050812,"(Glass Hanging Holder, Star Heart Wicker)"
71,0.066595,"(Glass Hanging Holder, Water Hot Bottle)"
72,0.060804,"(Way Sign Metal, Glass Hanging Holder)"


In [170]:
from mlxtend.frequent_patterns import association_rules

In [171]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.5) 

In [172]:
rules.sort_values(by='support', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
3,(Way Sign Metal),(Laundry Metal Sign),0.134211,0.123708,0.070228,0.523266,4.229831,1.0,0.053625,1.838113,0.881952,0.374168,0.455964,0.545479
4,(Laundry Metal Sign),(Way Sign Metal),0.123708,0.134211,0.070228,0.567692,4.229831,1.0,0.053625,2.002711,0.871381,0.374168,0.500677,0.545479
0,(Antique Wood White),(Glass Hanging Holder),0.115647,0.305041,0.063018,0.544919,1.786377,1.0,0.027741,1.527110,0.497774,0.176190,0.345168,0.375754
1,(Block Building Word),(Glass Hanging Holder),0.092256,0.305041,0.051209,0.555077,1.819677,1.0,0.023067,1.561975,0.496232,0.147966,0.359785,0.361477
2,(Star Heart Wicker),(Glass Hanging Holder),0.092937,0.305041,0.050812,0.546732,1.792320,1.0,0.022462,1.533217,0.487358,0.146361,0.347777,0.356653


In [173]:
product_groups_df[product_groups_df['Description New'].isin(['Antique Wood White','Glass Hanging Holder'])]

,Description Group,Description,Description New
262,229,HANGING HEART ZINC T-LIGHT HOLDER,Glass Hanging Holder
263,229,WHITE HANGING HEART T-LIGHT HOLDER,Glass Hanging Holder
264,229,WHITE TALL PORCELAIN T-LIGHT HOLDER,Glass Hanging Holder
265,229,RED HANGING HEART T-LIGHT HOLDER,Glass Hanging Holder
266,229,AGED GLASS SILVER T-LIGHT HOLDER,Glass Hanging Holder
267,229,SILVER HANGING T-LIGHT HOLDER,Glass Hanging Holder
268,229,BLACK LOVE BIRD T-LIGHT HOLDER,Glass Hanging Holder
269,229,PORCELAIN T-LIGHT HOLDERS ASSORTED,Glass Hanging Holder
270,229,WIRE FLOWER T-LIGHT HOLDER,Glass Hanging Holder
271,229,HANGING BAUBLE T-LIGHT HOLDER LARGE,Glass Hanging Holder
